## Grupo 4 - Laboratorio 5 - Python

Katiuska Olivera Quevedo (20172533) <br/> 
Rosemery Fernandez Sanchez (20172635) <br/> 
Aliro Cabrera Florez (20152034) <br/> 
Jose Uscamayta Quispe (20195674) <br/> 

In [1]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Penn = pd.read_csv("C:/Users/ok235/OneDrive/Documentos/GitHub/ECO224/Labs/data/penn_jae.dat" , sep='\s', engine='python')
n = Penn.shape[0]
#numero de covariables 
p_1 = Penn.shape[1]
Penn = Penn[ (Penn['tg'] == 4) | (Penn['tg'] == 0) ]

In [3]:
Penn.columns
Penn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5099 entries, 0 to 13911
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   abdt         5099 non-null   int64  
 1   tg           5099 non-null   int64  
 2   inuidur1     5099 non-null   int64  
 3   inuidur2     5099 non-null   int64  
 4   female       5099 non-null   int64  
 5   black        5099 non-null   int64  
 6   hispanic     5099 non-null   int64  
 7   othrace      5099 non-null   int64  
 8   dep          5099 non-null   int64  
 9   q1           5099 non-null   int64  
 10  q2           5099 non-null   int64  
 11  q3           5099 non-null   int64  
 12  q4           5099 non-null   int64  
 13  Unnamed: 13  5099 non-null   int64  
 14  q5           5099 non-null   int64  
 15  q6           5099 non-null   int64  
 16  recall       5099 non-null   int64  
 17  agelt35      5099 non-null   int64  
 18  agegt54      5099 non-null   int64  
 19  durab

In [4]:
# Dependent variable
Penn['T4'] = (Penn[['tg']]==4).astype(int)

# Create category variable
Penn['dep'] = Penn['dep'].astype( 'category' )
Penn.head()


,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,0,0,1,0,NaN,0
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
11,10607,4,9,9,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,NaN,1
12,10831,0,27,27,0,0,0,0,1,0,...,0,0,1,1,0,1,0,0,NaN,0


In [5]:
Penn['dep'].unique()

[2, 0, 1]
Categories (3, int64): [2, 0, 1]

In [6]:
Penn.columns

Index(['abdt', 'tg', 'inuidur1', 'inuidur2', 'female', 'black', 'hispanic',
       'othrace', 'dep', 'q1', 'q2', 'q3', 'q4', 'Unnamed: 13', 'q5', 'q6',
       'recall', 'agelt35', 'agegt54', 'durable', 'nondurable', 'lusd', 'husd',
       'muld', 'T4'],
      dtype='object')

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import numpy as np

In [8]:
Penn["inuidur1"] = np.log (Penn["inuidur1"])


In [9]:
model =  "inuidur1~T4+ (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)"
model_results = smf.ols( model , data = Penn ).fit().get_robustcov_results(cov_type = "HC1")

In [10]:
print(model_results.summary())
print( "Number of regressors in the basic model:",len(model_results.params), '\n')

                            OLS Regression Results                            
Dep. Variable:               inuidur1   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     15.32
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           6.43e-42
Time:                        03:20:39   Log-Likelihood:                -8128.2
No. Observations:                5099   AIC:                         1.629e+04
Df Residuals:                    5082   BIC:                         1.640e+04
Df Model:                          16                                         
Covariance Type:                  HC1                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.7723      0.050     35.154      

In [11]:
coef1 =model_results.summary2().tables[1]['Coef.']['T4']
err1 =model_results.summary2().tables[1]['Std.Err.']['T4']

In [12]:
coef2 =model_results.summary2().tables[1]['Coef.']['female']
err2 =model_results.summary2().tables[1]['Std.Err.']['female']

In [13]:
coef3 =model_results.summary2().tables[1]['Coef.']['black']
err3 =model_results.summary2().tables[1]['Std.Err.']['black']

#### Real Data 

We extract the real coef and std err of the variables (T4, female, and black)

In [14]:
df=pd.DataFrame({"variable":['T4','female','black'], 
                 "coef":[coef1,coef2,coef3],
                 "err":[err1,err2,err3]}) 
df

,variable,coef,err
0,T4,-0.076206,0.035211
1,female,0.138128,0.034827
2,black,-0.307905,0.059723


## Bootstraping 

In [15]:
def get_indices(data,num_samples):
    return  np.random.choice(data.index, num_samples, replace=True)


In [16]:
def get_estimates(data,index):
    data_1 = data.loc[index]

    model = "inuidur1~T4+ (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)"

    model_results = smf.ols( model , data = data_1 ).fit().get_robustcov_results(cov_type = "HC1")

    T4_coef = model_results.summary2().tables[1]['Coef.']['T4']
    female_coef = model_results.summary2().tables[1]['Coef.']['female']
    black_coef = model_results.summary2().tables[1]['Coef.']['black']

    return [T4_coef,female_coef, black_coef]


In [17]:
def boot(data,func,R):
    T4 = []
    female = []
    black = []
    for i in range(R):
        T4.append(func(data,get_indices(data,5099))[0])
        female.append(func(data,get_indices(data,5099))[1]) 
        black.append(func(data,get_indices(data,5099))[2]) 
    T4_statistics = {'estimated_value':np.mean(T4),'std_error':np.std(T4)}   
    female_statistices = {'estimated_value':np.mean(female),'std_error':np.std(female)}   
    black_statistices = {'estimated_value':np.mean(black),'std_error':np.std(black)} 
    return {'T4':T4_statistics,'female_statistices':female_statistices,'black_statistices':black_statistices}


In [18]:
results =boot(Penn,get_estimates,1000)

In [19]:
results

{'T4': {'estimated_value': -0.07538444679606028,
  'std_error': 0.034736600309284736},
 'female_statistices': {'estimated_value': 0.14019048407111312,
  'std_error': 0.035352948234779476},
 'black_statistices': {'estimated_value': -0.3076308058390237,
  'std_error': 0.05955396376021122}}

In [20]:
results['T4']['estimated_value']

-0.07538444679606028

In [21]:
coef1 =model_results.summary2().tables[1]['Coef.']['T4']
err1 =model_results.summary2().tables[1]['Std.Err.']['T4']

In [22]:
coef2 =model_results.summary2().tables[1]['Coef.']['female']
err2 =model_results.summary2().tables[1]['Std.Err.']['female']

In [23]:
coef3 =model_results.summary2().tables[1]['Coef.']['black']
err3 =model_results.summary2().tables[1]['Std.Err.']['black']

In [24]:
coef5 =results['T4']['estimated_value']
err5 =results['T4']['std_error']

In [25]:
coef6 =results['female_statistices']['estimated_value']
err6 =results['female_statistices']['std_error']

In [26]:
coef7 =results['black_statistices']['estimated_value']
err7 =results['black_statistices']['std_error']

### Bootstraping data

We extract the real coef and std err of the variables (T4, female, and black)

In [27]:
df=pd.DataFrame({"variable":['T4','female','black','T4boot','femaleboot','blackboot'], 
                 "coef":[coef1,coef2,coef3,coef5,coef6,coef7],
                 "err":[err1,err2,err3,err5,err6,err7]}) 
df

,variable,coef,err
0,T4,-0.076206,0.035211
1,female,0.138128,0.034827
2,black,-0.307905,0.059723
3,T4boot,-0.075384,0.034737
4,femaleboot,0.140190,0.035353
5,blackboot,-0.307631,0.059554


The difference between the coefficient of the real data and the bootstrapping data is minimal, this can be explained by the high number of replicas (1000) with replacement of the data. 